Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Ariel Steele, Zihan Xu, and Sarthak Chaturvedi"

# <u>Team Homework</u>

CEE 8813-F & ISYE 8803-PTA Transportation Network Modeling and Analysis

**Due: Nov 30, 2021**

## Map of Network

![SiouxFallsMap_AAA1998.jpg](SiouxFallsMap_AAA1998.jpg)

Import network and codes.

In [2]:
from node import Node
from link import Link
from path import Path
from od import OD
from network import Network
from nose.tools import assert_equal

import sys
import traceback
import utils
import random
import numpy as np
from scipy.linalg import solve
from scipy.optimize import minimize

FRANK_WOLFE_STEPSIZE_PRECISION = 1e-7
UE_PRECISION = 10

In [3]:
net = Network("SiouxFalls_net.tntp", "SiouxFalls_trips.tntp")

We already have access to the previous tools created, including calculateCost for links (using BPR), calculateBeckmannComponent (necessary for BPR, not sure if we still need it), shiftFlows (gets used in the UE solution), shortestPath (used for all or nothing assignment), allOrNothing (gives us the initial all or nothing assignment), FrankWolfeStepSize (also necessary for UE solution), and userEquilibriumFW (solves the UE STA). We should take advantage of these functions but will likely need to adjust them, especially the cost function.

## Scenario 1: All HDV on Existing Network

This should simply be the solution we get from running the pre-existing userEquilibriumFW.

In [4]:
for ij in net.link:
    print(ij)
    print(net.link[ij].flow)

(1,2)
0
(1,3)
0
(2,1)
0
(2,6)
0
(3,1)
0
(3,4)
0
(3,12)
0
(4,3)
0
(4,5)
0
(4,11)
0
(5,4)
0
(5,6)
0
(5,9)
0
(6,2)
0
(6,5)
0
(6,8)
0
(7,8)
0
(7,18)
0
(8,6)
0
(8,7)
0
(8,9)
0
(8,16)
0
(9,5)
0
(9,8)
0
(9,10)
0
(10,9)
0
(10,11)
0
(10,15)
0
(10,16)
0
(10,17)
0
(11,4)
0
(11,10)
0
(11,12)
0
(11,14)
0
(12,3)
0
(12,11)
0
(12,13)
0
(13,12)
0
(13,24)
0
(14,11)
0
(14,15)
0
(14,23)
0
(15,10)
0
(15,14)
0
(15,19)
0
(15,22)
0
(16,8)
0
(16,10)
0
(16,17)
0
(16,18)
0
(17,10)
0
(17,16)
0
(17,19)
0
(18,7)
0
(18,16)
0
(18,20)
0
(19,15)
0
(19,17)
0
(19,20)
0
(20,18)
0
(20,19)
0
(20,21)
0
(20,22)
0
(21,20)
0
(21,22)
0
(21,24)
0
(22,15)
0
(22,20)
0
(22,21)
0
(22,23)
0
(23,14)
0
(23,22)
0
(23,24)
0
(24,13)
0
(24,21)
0
(24,23)
0


In [5]:
net.userEquilibriumFW()

0.23900407552719116 ==
Iteration 1: gap 4.464857; obj fun 9031515.396921
0.167308509349823 ==
Iteration 2: gap 1.330809; obj fun 7331935.185589
0.17806977033615112 ==
Iteration 3: gap 0.545168; obj fun 6554032.761622
0.20733028650283813 ==
Iteration 4: gap 0.413954; obj fun 6108093.401142
0.21369165182113647 ==
Iteration 5: gap 0.352965; obj fun 5736180.831744
0.16233962774276733 ==
Iteration 6: gap 0.264349; obj fun 5519688.979265
0.1841183304786682 ==
Iteration 7: gap 0.212625; obj fun 5337258.725508
0.13067513704299927 ==
Iteration 8: gap 0.178201; obj fun 5217619.935518
0.12838858366012573 ==
Iteration 9: gap 0.154971; obj fun 5132675.105711
0.13419324159622192 ==
Iteration 10: gap 0.156196; obj fun 5045905.633318
0.0966760516166687 ==
Iteration 11: gap 0.114622; obj fun 4991304.118794
0.2756773829460144 ==
Iteration 12: gap 0.189011; obj fun 4842419.414871
0.08268529176712036 ==
Iteration 13: gap 0.091959; obj fun 4808099.980289
0.18272536993026733 ==
Iteration 14: gap 0.106719; o

Flows for Scenario 1.

In [6]:
for ij in net.link:
    print(ij)
    print(net.link[ij].flow)

(1,2)
7727.692697173979
(1,3)
11388.846750920227
(2,1)
7721.932809718441
(2,6)
6738.204407161642
(3,1)
11394.606638375762
(3,4)
17675.35862640572
(3,12)
14819.293891155065
(4,3)
17646.163913414643
(4,5)
18809.440026905402
(4,11)
6400.23856827053
(5,4)
18966.588316593705
(5,6)
6908.617539253575
(5,9)
17061.808133279294
(6,2)
6732.444519706108
(6,5)
6958.601205823046
(6,8)
12504.697496081808
(7,8)
13370.008714675956
(7,18)
16633.764079872253
(8,6)
12548.921275195755
(8,7)
13331.080035730689
(8,9)
7623.074539977357
(8,16)
7904.9233021173995
(9,5)
17168.972756398143
(9,8)
7612.696118816826
(9,10)
21685.29157426339
(10,9)
21882.0777762217
(10,11)
17450.659729533785
(10,15)
23350.012734088745
(10,16)
10694.744132784464
(10,17)
8383.409509430048
(11,4)
6313.895565591152
(11,10)
17408.96447488131
(11,12)
7261.516294635895
(11,14)
9456.844045269057
(12,3)
14854.248491601678
(12,11)
7247.630870343936
(12,13)
15749.54516272333
(13,12)
15870.614338877984
(13,24)
10560.469558587187
(14,11)
9442.691

## Scenario 2: Half HDV Half AV on Existing Network

Confused on how to handle this, but I can think of two approaches. We could compute the link costs for HDVs and AVs at a 50/50 mix and then average those two costs for each link and assign the result as the overall link cost. However, that seems like  an oversimplification. Or, we could split the demand in half and then perform the STA twice, once for HDV and once for AV. However, this also has a problem because it ignores any interaction between HDV and AV. Please share your ideas.

$Q_{a,A}=N_a v_a\frac{1}{v_a \Delta t_A + l}$

$Q_{a,H}=N_a v_a\frac{1}{v_a \Delta t_H + l}$

$N_a = 2$ lanes

$v_a = 40$ mph $= 17.882$ m/s

$t_A = 0.75$ s

$t_H = 1.5$ s

$l = 4.5$ m

In [7]:
QaA = (2*17.882)/((17.882*0.75)+4.5)
QaH = (2*17.882)/((17.882*1.5)+4.5)

print('Link capacity for AV:')
print(QaA)
print('Link capacity for HDV:')
print(QaH)

Link capacity for AV:
1.9967060268542558
Link capacity for HDV:
1.1417808000510807


$c_{a,H}(x_{a,H},x_{a,A}) = \frac{l_a}{v_a} [1 + (\frac{x_{a,H}}{Q_{a,H}} + \frac{x_{a,A}}{Q_{a,A}})^4]$

$c_{a,A}(x_{a,H},x_{a,A}) = \frac{l_a}{v_a} [1 + (\frac{x_{a,H}}{Q_{a,H}} + \frac{x_{a,A}}{Q_{a,A}})^4]$

In [8]:
for ij in net.link:
    caH = ((net.link[ij].length)/17.882) * (1 + (net.link[ij].flow/QaH + net.link[ij].flow/QaA))
    caA = ((net.link[ij].length)/17.882) * (1 + (net.link[ij].flow/QaH + net.link[ij].flow/QaA))

This is a start. Now I think we would have to adapt one of the previous functions to update the cost. We also need to differentiate between HDV flows and AV flows, but I just used the previous flows for now as a starting place.

## Scenario 3: Half HDV Half AV on Existing Network with Added AV-Dedicated Lane

This one will be mostly the same as Scenario 2, but we need to change the capacity and link cost for '(10,15)' and '(15,10)'. I think the easiest way to model this would be to essentially split these 2 links with 2 lanes each into 4 links with 1 lane each. '(10,15)M', '(15,10)M', '(10,15)A', and '(15,10)A' where M is for mixed flow and A is for AV dedicated.

## Scenario 4: ???

Need to decide on an additional scenario here. All of the suggestions they gave sound interesting and doable, so feel free to share your opinions. Changing the reaction time for AVs or changing the location of the dedicated lane seem easiest to implement.

## Analysis

List of Changes Made to Single-Class STA to Extend to Multi-Class

Comparison of Performance Across Scenarios

Discussion of Fourth Scenario